# Ch3. word2vec

### Recap [Ch 2. 단어의 분산 표현]

- 통계 기반 기법 : 단어의 빈도를 가지고 표현
- **추론 기반 기법**

### 이번 장에서 배울 내용

- 추론 기반 기법은 추측하는 것이 목적이며, 그 부산물로 단어의 분산 표현을 얻을 수 있다.
- word2vec은 추론 기반 기법이며, 단순한 2층 신경망이다.
- word2vec은 skip-gram 모델과 CBOW 모델을 제공한다.
- CBOW 모델은 여러 단어(맥락)로부터 하나의 단어(타깃)를 추측한다.
- 반대로 skip-gram 모델은 하나의 단어(타깃)로부터 다수의 단어(맥락)을 추측한다.
- word2vec은 가중치를 다시 학습할 수 있으므로, 단어의 분산 표현 갱신이나 새로운 단어 추가를 효율적으로 수행할 수 있다.

### 추론 기반 기법이란?
   - 추론 하는 기법
   - 대표적으로 신경망을 이용한 **word2vec** 이 이에 해당한다.

### 이번장에서 다룰 내용

- word2vec의 구조
- '단순한' word2vec 구현
    - Ch4에서 '진짜' word2vec을 완성할 예정


## 3.1 추론 기반 기법과 신경망

단어를 벡터로 표현하는 방법은 지금까지 활발히 연구되었으며, 그중에서도 성공적인 기법들은 크게 두 부류로 나눌 수 있다.

- **통계 기반 기법**
- **추론 기반 기법**

차이점 : 단어의 의미를 얻는 방식  
공통점 : 분포 가설을 가짐 ?? 어떤 의미인가

 이번 절에서는 통계 기반 기법의 문제를 지적하고, 그 대안인 추론 기반 기법의 이점을 거시적인 관점에서 설명한다. 그 이후 `word2vec`의 전처리를 위해 신경망으로 '단어'를 처리하는 예를 확인해보자.

### 3.3.1 통계 기반 기법의 문제점

#### 계산 복잡도

통계 기반 기법은 주변 단어의 빈도를 기초로 단어를 표현했다. 구체적으로는 단어의 동시발생 행렬을 만들고 그 행렬을 만들고 그 행렬에 **SVD**를 적용하여 밀집벡터(단어의 분산 표현)를 얻었다. 그러나 이 방식은 대규모 말뭉치를 다룰 때 문제가 발생한다.

현실에서 다루는 말뭉치의 어휘 수는 무수히 많다. 예를 들어 영어의 어휘 수는 100만을 넘는다고 하는데, 통계 기반 기법에서는 형상이 `100만 x 100만`인 행렬을 만들게 된다. 이런 거대 행렬에 **SVD** 기법을 적용하는 것은 현실적이지 않다.

> **NOTE**  
> SVD를 $n \times n$ 행렬에 적용하는 비용은 $Q(n^{n})$이다. 슈퍼컴퓨터로 동원해도 처리할 수 없는 수준이라고 한다. 근사적인 기법?과 희소행렬의 성질 등을 이용해 속도를 개선할 수 있다고 한다. 하지만 여전히 상당한 컴퓨팅 자원을 들여 장시간 계산해야 한다.



### 3.1.2 추론 기반 기법 개요

[그림 3-2] 처럼 주변 단어(맥락)가 주어졌을 때 "?"에 무슨 단어가 들어가는지 추측하는 작업

**[그림 3-2]** 주변 단어들을 맥락으로 사용해 "?"에 들어갈 단어를 추측한다.

그림...

위처럼 추론 문제를 풀고 학습하는 것이 '추론 기반 기법'이 다루는 문제이다. 이러한 추론 문제를 계속해서 반복하면서 단어의 출현 패턴을 학습하는 것이다. '모델 관점'에서 보면, 이 추론 문제는 **[그림 3-3]**처럼 보인다.

**[그림 3-3]** 추론 기반 기법 : 맥락을 입력하면 모델은 각 단어의 출현 확률을 출력한다.

위 처럼 추론 기반 기법에는 어떠한 모델이 등장한다. 이 모델로 신경망을 사용하며, 모델은 맥락 정보를 입력받아 (출현할 수 있는) 각 단어의 출현 확률을 출력한다. 

> **NOTE**  
> 추론 기반 기법도 통계 기반 기법처럼 분포 가설에 기초하게 된다. **분포 가설이란?** **"단어의 의미는 주변 단어에 의해 형성된다."**는 가설로, 추론 기반 기법에서는 이를 앞에서와 같은 추측 문제로 귀결시켰다. 이 처럼 두 기법 모두 분포 가설 근거하는 **'단어의 동시발생 가능성'**을 얼마나 잘 모델링 하는가 중요한 연구 주제이다.

### 3.1.3 신경망에서의 단어 처리

신경망을 이용해 '단어'를 처리한다. 하지만, 신경망은 'you'와 'say' 등의 단어를 있는 그대로 처리할 수 없으니 단어를 '고정 길이의 벡터'로 변환해야 한다. 이때 사용하는 방법이 단어를 **원핫 표현(원핫 벡터)**으로 변환하는 것이다.

**[그림 3-4]** : 단어, 단어 ID, 원핫 표현

이를 통해 얻는 이점은 신경망의 입력층을 **[그림 3-5]**처럼 뉴런의 수를 **"고정"** 할 수 있다.

**[그림 3-5]** : 입력층의 뉴런 : 각 뉴런이 각 단어에 대응(해당 뉴런이 1이면 파랑색, 0이면 회색)

**[그림 3-6]** : 완전연결계층에 의한 변환 : 입력층의 각 뉴런은 7개의 단어 각각에 대응(은닉층 뉴런은 3개를 준비함)

**[그림 3-7]** : 완전연결계층에 의한 변환을 단순화한 그림(완전연결계층의 가중치를 7 x 3 크기의 **W**라는 행렬로 표현)

In [5]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])  # 입력
W = np.random.randn(7, 3)              # 가중치
h = np.matmul(c, W)                    # 중간노드
print(h)

[[-0.30620566  0.135077   -0.81835433]]


> **Warning**  
> 이 코드에서는 입력 데이터(c)의 차원 수(ndim)은 2이다. 이는 미니배치 처리를 고려한 것으로 최초의 차원(0번째 차원)에 각 데이터를 저장한다.

In [9]:
np.ndim(c)

2

**[그림 3-8]** : 맥락 **c**와 **W**의 곱으로 해당 위치의 행벡터가 추출된다. (각 요소의 가중치 크기는 흑백의 진하기로 표현)

In [14]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[ 1.53348768 -1.09848979 -1.1196359 ]]


## 3.2 단순한 word2vec

- word2vec 구현!

**[그림 3-3]**의 '모델'을 신경망으로 구축해보자.

In [15]:
[]

[]

 이번절에서는 **word2vec**에서의 **CBOW(continuous bag-of-words)** 모델이다.
 
 > **Warning**  
 > word2vec이라는 용어는 원래 프로그램이나 도구를 가리키는 데 사용됐습니다. 그런데 이 용어가 유명해지면서, 문맥에 따라서는 신경망 모델을 가리키는 경우도 많이 볼 수 있습니다. **CBOW** 모델과 **Skip-gram** 모델은 **word2vec**에서 사용되는 신경망입니다. 이번 절에서는 **CBOW** 모델을 중심으로 이야기를 풀어가며, 두 모델의 차이는 '3.5.2. skip-gram 모델'절에서 자세히 설명하겠습니다.

### 3.2.1 CBOW 모델의 추론 처리

맥락으로부터 **Target**을 추측하는 용도의 신경망이다. ('타깃'은 중앙 단어이고 그 주변 단어들이 '맥락'입니다.). 이 **CBOW** 모델이 가능한 한 정확하게 추론하도록 훈련시켜서 단어의 분산 표현을 얻어낼 것이다.

**CBOW** 모델의 입력은 맥락입니다. 
- 맥락 : "you", "goodbye"
- 타겟 : ?

**[그림 3-9]** **CBOW**모델의 신경망 구조

> **Warning**  
> 이 그림에서 입력층이 2개인 이유는 맥락으로 고려할 단어를 2개로 정했기 때문이다. 즉, 맥락에 포함시킬 단어가 $N$개라면 입력층도 $N$가 된다.

입력이 여러 개이면 전체를 **평균**하면 된다. 앞의 예에 대입해보면 다음과 같다. 완전연결계층에 의한 첫 번째 입력층이 $h_1$으로 변환되고, 두 번째 입력층이 $h_2$로 변환되었다고 하면, 은닉층 뉴런은 $\frac{1}{2}(h_1 + h_2)$가 되는 것이다.

마지막으로  **[그림 3-9]**의 출력층을 보면, 출력층의 뉴런은 총 7개인데, 여기서 중요한 것은 이 뉴런 하나하나가 각각의 단어에 대응한다는 것이다. 그리고 출력층 뉴런은 각 던어의 **'점수'**를 뜻하며, 값이 높을수록 대응 단어의 출현 확률도 높아진다. 여기서 점수란 확률로 해석되기 전의 값이고, 이 점수에 소프트맥스 함수를 적용해서 '확률'로 얻을 수 있다.

> **Warning**  
> 점수를 **Softmax** 계층에 통과시킨 후의 뉴런을 '출력층'이라고도 한다. 교재에서는 점수를 출력하는 노드를 '출력층'이라고 하자.